TRANSFORMACIONES:

    - Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

    - Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

    - De haber fechas, deberán tener el formato AAAA-mm-dd

    - Los campos de texto deberán estar en minúsculas, sin excepciones

    - El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)


CARGA DE DATOS
(solo se carga uno a la vez)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Datasets_original/amazon_prime_titles-score.csv')
platform = 'a'

In [ ]:
df = pd.read_csv('Datasets_original/disney_plus_titles-score.csv')
platform = 'd'

In [ ]:
df = pd.read_csv('Datasets_original/hulu_titles-score (2).csv')
platform = 'h'

In [ ]:
df = pd.read_csv('Datasets_original/netflix_titles-score.csv')
platform = 'n'

CAMPO ID

In [ ]:
df['id'] = platform + df['show_id']#remplazar el valor dentro de '' por la primera letra de la plataforma

NULOS EN RATING

In [ ]:
df['rating'].fillna('G', inplace=True)

FECHAS

In [ ]:
#creamos un metodo para cambiar la fecha al formato pedido
def dateTransform(date: str) -> int :
     date = date.replace(', ',' ')#eliminamos la coma
     dateSplit = date.split()#dividimos la fecha que queda ['mes', dia, año]

     #se saca el numero del mes deacuerdo con el nombre
     match dateSplit[0]:
          case 'January':
               monthNum =  1
          case 'February':
               monthNum =  2
          case 'March':
               monthNum =  3
          case 'April':
               monthNum =  4
          case 'May':
               monthNum =  5
          case 'June':
               monthNum =  6
          case 'July':
               monthNum =  7
          case 'August':
               monthNum =  8
          case 'September':
               monthNum =  9
          case 'October':
               monthNum =  10
          case 'November':
               monthNum =  11
          case 'December':
               monthNum =  12
          case _:
               monthNum =  0

     #rearmamos la fecha con el formato pedido y la retornamos
     date = dateSplit[2] + '-' + str(monthNum) + '-' + dateSplit[1]
     return date
     
#aplicamos el metodo anterior a todas las fechas
df['date_added'] = df['date_added'].apply( lambda date: dateTransform(date) if isinstance(date, str) else date )


MINUSCULAS

In [ ]:
columns = ['id','show_id','type','title','director','cast','country','date_added','release_year','rating','duration','listed_in','description','score']

for i in columns:
    df[i] = df[i].apply( lambda txt: txt.lower() if isinstance(txt, str) else txt )


CAMPOS DURACION

In [ ]:
#dividimos duration en duration_int y duration_type
duration = df['duration'].str.split(' ', expand=True)
df['duration_int'] = duration[0]
df['duration_type'] = duration[1]

#normalisamos seasson en duration_type
df['duration_type'] = df['duration_type'].apply( lambda d_type: 'season' if d_type == 'seasons' else d_type )

GUARDAMOS

In [ ]:
#ordenamos las columnas
df = df.loc[:,['id','show_id','type','title','director','cast','country','date_added','release_year','rating','duration_int','duration_type','listed_in','description','score']]

#agregamos el dataset limpio al archivo nuevo
df.to_csv('Datasets_Limpio/platformsTitles.csv', sep=';', mode='a', index=False, header=False)